# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

City_ID               City      Lat      Lng  Max Temp  Humidity  \
0        0       Kudahuvadhoo   2.6708  72.8944     81.75        75   
1        1  Port-Aux-Francais -49.3500  70.2167     36.10        73   
2        2          Jamestown  42.0970 -79.2353     48.02        87   
3        3            Grabouw -34.1515  19.0151     59.40        90   
4        4           At-Bashi  41.1702  75.8106     40.71        64   

   Cloudiness  Wind Speed Country        Date  
0          12        9.78      MV  1697587582  
1          96       13.91      TF  1697587582  
2         100        5.75      US  1697587245  
3          10        5.41      ZA  1697587583  
4          70        5.97      KG  1697587583

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 500,
    frame_height = 400,
    size = "Humidity",
    color= "City"
)

# Display the map
map_plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df.loc[city_data_df["Max Temp"]< 80]
city_data_df = city_data_df.loc[city_data_df["Max Temp"]> 70]
city_data_df = city_data_df.loc[city_data_df["Cloudiness"]<20]
city_data_df = city_data_df.loc[city_data_df["Humidity"]<70]

# Drop any rows with null values
city_data_df.dropna(axis=0, how="any")
city_data_df["City"]=city_data_df["City"].str.title()

# Display sample data
city_data_df.head(10)


City_ID            City      Lat       Lng  Max Temp  Humidity  \
35        35      San Angelo  31.4638 -100.4370     79.02        26   
91        91         Awjilah  29.1081   21.2869     70.02        49   
110      110   Alice Springs -23.7000  133.8833     72.95        14   
132      132            Naze  28.3667  129.4833     70.81        56   
173      173         Turabah  21.2141   41.6331     73.45        45   
237      237       Carnarvon -24.8667  113.6333     76.06        36   
240      240         Balotra  25.8333   72.2333     78.48        60   
245      245     Al Fallujah  33.3538   43.7795     72.88        40   
251      251  Guerrero Negro  27.9769 -114.0611     72.99        65   
261      261    Spring Creek  40.7266 -115.5859     70.81        21   

     Cloudiness  Wind Speed Country        Date  
35            0        6.91      US  1697587593  
91            0        7.52      LY  1697587611  
110           0       11.50      AU  1697587617  
132           0        5.35      JP  1697587624  
173           0        5.10      SA  1697587637  
237           0       20.36      AU  1697587655  
240           6        3.20      IN  1697587656  
245           0        1.41      IQ  1697587658  
251          16       17.00      MX  1697587439  
261           0        8.66      US  1697587663

### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df.copy()
hotel_df = hotel_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""

# Display sample data
hotel_df.head(10)

City Country      Lat       Lng  Humidity Hotel Name
35       San Angelo      US  31.4638 -100.4370        26           
91          Awjilah      LY  29.1081   21.2869        49           
110   Alice Springs      AU -23.7000  133.8833        14           
132            Naze      JP  28.3667  129.4833        56           
173         Turabah      SA  21.2141   41.6331        45           
237       Carnarvon      AU -24.8667  113.6333        36           
240         Balotra      IN  25.8333   72.2333        60           
245     Al Fallujah      IQ  33.3538   43.7795        40           
251  Guerrero Negro      MX  27.9769 -114.0611        65           
261    Spring Creek      US  40.7266 -115.5859        21

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "filter": "",
    "limit": 1,
    "bias": "",
    "lang": "en",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lon = row["Lng"]

 
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
    
# Display sample data
hotel_df

Starting hotel search
San Angelo - nearest hotel: Hilton Hotel
Awjilah - nearest hotel: No hotel found
Alice Springs - nearest hotel: Aurora Alice Springs
Naze - nearest hotel: ビジネス旅館畠山
Turabah - nearest hotel: No hotel found
Carnarvon - nearest hotel: No hotel found
Balotra - nearest hotel: No hotel found
Al Fallujah - nearest hotel: عمارة اطلس
Guerrero Negro - nearest hotel: Plaza sal paraiso
Spring Creek - nearest hotel: No hotel found
Gallup - nearest hotel: No hotel found
Sao Francisco - nearest hotel: Hotel Atalaia
Tlahualilo De Zaragoza - nearest hotel: No hotel found
Goondiwindi - nearest hotel: Queensland Hotel
Prineville - nearest hotel: Best Western
Dhrangadhra - nearest hotel: Aaram
Ma'Rib - nearest hotel: No hotel found
Gebeit - nearest hotel: No hotel found
Nahiyat Ash Shinafiyah - nearest hotel: No hotel found
Al Kharijah - nearest hotel: Kharga Hotel
Barika - nearest hotel: فندق ميساوي
Silver City - nearest hotel: Murray Hotel
Crane - nearest hotel: No hotel found


City Country      Lat       Lng  Humidity  \
35               San Angelo      US  31.4638 -100.4370        26   
91                  Awjilah      LY  29.1081   21.2869        49   
110           Alice Springs      AU -23.7000  133.8833        14   
132                    Naze      JP  28.3667  129.4833        56   
173                 Turabah      SA  21.2141   41.6331        45   
237               Carnarvon      AU -24.8667  113.6333        36   
240                 Balotra      IN  25.8333   72.2333        60   
245             Al Fallujah      IQ  33.3538   43.7795        40   
251          Guerrero Negro      MX  27.9769 -114.0611        65   
261            Spring Creek      US  40.7266 -115.5859        21   
272                  Gallup      US  35.5281 -108.7426        14   
280           Sao Francisco      BR -15.9486  -44.8644        28   
318  Tlahualilo De Zaragoza      MX  26.1167 -103.4500        30   
326             Goondiwindi      AU -28.5500  150.3167        34   
330              Prineville      US  44.2999 -120.8345        30   
371             Dhrangadhra      IN  22.9833   71.4667        59   
393                  Ma'Rib      YE  15.4591   45.3253        23   
402                  Gebeit      SD  21.0667   36.3167        47   
424  Nahiyat Ash Shinafiyah      IQ  31.5874   44.6602        35   
472             Al Kharijah      EG  25.4514   30.5464        41   
535                  Barika      DZ  35.3890    5.3658        23   
556             Silver City      US  32.7701 -108.2803        38   
557                   Crane      US  31.3974 -102.3501        27   

               Hotel Name  
35           Hilton Hotel  
91         No hotel found  
110  Aurora Alice Springs  
132              ビジネス旅館畠山  
173        No hotel found  
237        No hotel found  
240        No hotel found  
245            عمارة اطلس  
251     Plaza sal paraiso  
261        No hotel found  
272        No hotel found  
280         Hotel Atalaia  
318        No hotel found  
326      Queensland Hotel  
330          Best Western  
371                 Aaram  
393        No hotel found  
402        No hotel found  
424        No hotel found  
472          Kharga Hotel  
535           فندق ميساوي  
556          Murray Hotel  
557        No hotel found

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 500,
    frame_height = 300,
    color = "City",
    hover_cols=["City", "Hotel Name"]
)

# Display the map
map_plot




:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name)